In [1]:
import os
import sys
sys.path.append('../../source')
sys.path.append('../../source/maskrcnn')
sys.path.append('../../source/dataset')
sys.path.append('../../source/miq')

sys.path.append('C:\\Program Files\\Micro-Manager-2.0beta')

import MMCorePy
import sc_utils
import scipy.misc
import alignment
import focus
import chip
import position as pos

import tifffile as tif

save_dir = 'focus_images'
os.makedirs(save_dir, exist_ok=True)

mmc = sc_utils.get_stage_controller(cfg='../../../config/scope_stage.cfg')

Using TensorFlow backend.


In [26]:
curr_chip = chip.ML_Chip()
chip_name = "ML_Chip3_no_fluid"


x,y,z = pos.current(mmc)
p1 = pos.StagePosition(x=x, y=y)
p2 = pos.StagePosition(x=x-curr_chip.CHIP_WIDTH, y=y)
p3 = pos.StagePosition(x=x-curr_chip.CHIP_WIDTH, y=y-curr_chip.CHIP_HEIGHT)

corners_pl = pos.PositionList(positions=[p1,p2,p3])
curr_chip.initalize(corners_pl)
focus_points = curr_chip.get_focus_position_list(5,4)


pos_list = pos.PositionList()
for i, posit in enumerate(focus_points):
    pos.set_pos(mmc, x=posit.x, y=posit.y)
    input ("Press ENTER when focused...")
    sp = pos.StagePosition(x=posit.x, y=posit.y, z=mmc.getPosition())
    pos_list.append(sp)

imaging_pl = curr_chip.get_position_list(pos_list)

Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 
Press ENTER when focused... 


In [27]:
imaging_pl.save('imaging_pl_ML_3','.')

In [28]:
cam = sc_utils.start_cam()
for i in range (11):
    offset = i * 10
    direction = 1
    os.makedirs('offset_'+str(offset), exist_ok=True)
    for i, posit in enumerate(imaging_pl):
        pos.set_pos(mmc, posit.x, posit.y, z=posit.z+offset*direction)
        frame = cam.get_frame(exp_time=1).reshape(cam.sensor_size[::-1])
        frame = focus.bytescale(frame)
        tif.imwrite('offset_'+str(offset)+'/'+chip_name+'-'+str(i)+'.tif', frame)
        direction = direction*-1
sc_utils.close_cam(cam)